# Module 3. Homework: Classification

In [241]:
import numpy as np
import pandas as pd

In [242]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


## Data preparation

In [243]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [244]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [245]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
y_col = ['converted']

In [246]:
df[categorical] = df[categorical].fillna('NA')

In [247]:
df[numerical] = df[numerical].fillna(0.0)

In [248]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

## Question 1
What is the most frequent observation (mode) for the column `industry`?

In [249]:
df.industry.mode()

0    retail
Name: industry, dtype: object

## Question 2
Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your dataset.
In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

- `interaction_count` and `lead_score`
- `number_of_courses_viewed` and `lead_score`
- `number_of_courses_viewed` and `interaction_count`
- `annual_income` and `interaction_count`

Only consider the pairs above when answering this question.

In [250]:
for c in numerical:
    print(c)
    print(df[numerical].corrwith(df[c]).abs().sort_values(ascending=False))
    print()

number_of_courses_viewed
number_of_courses_viewed    1.000000
interaction_count           0.023565
annual_income               0.009770
lead_score                  0.004879
dtype: float64

annual_income
annual_income               1.000000
interaction_count           0.027036
lead_score                  0.015610
number_of_courses_viewed    0.009770
dtype: float64

interaction_count
interaction_count           1.000000
annual_income               0.027036
number_of_courses_viewed    0.023565
lead_score                  0.009888
dtype: float64

lead_score
lead_score                  1.000000
annual_income               0.015610
interaction_count           0.009888
number_of_courses_viewed    0.004879
dtype: float64




number_of_courses_viewed  
interaction_count           0.023565  
lead_score                  0.004879  

**annual_income**  
**interaction_count           0.027036**  

interaction_count  
lead_score                  0.009888  

## Split the data

In [251]:
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

ys = []
for d in [df_full_train, df_train, df_val, df_test]:
    d.reset_index(inplace=True, drop=True)
    col = 'converted'
    ys.append(d[col].values)
    del d[col]

y_full_train, y_train, y_val, y_test = ys
del ys

## Question 3
- Calculate the mutual information score between `converted` and other categorical variables in the dataset. Use the training set only.
- Round the scores to 2 decimals using `round(score, 2)`.

Which of these variables has the biggest mutual information score?

In [252]:
from sklearn.metrics import mutual_info_score

def mutual_info_converted_score(series):
    return mutual_info_score(series, y_train)

mi = df_train[categorical].apply(mutual_info_converted_score)
mi.sort_values(ascending=False)

lead_source          0.035396
employment_status    0.012938
industry             0.011575
location             0.004464
dtype: float64

## Question 4
- Now let's train a logistic regression.
- Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
- Fit the model on the training dataset.
  - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
  - `model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)`
- Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [253]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(df_val[categorical + numerical].to_dict(orient='records'))

In [254]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict_proba(X_val)[:, 1]
decisions = (y_pred >= 0.5)
(y_val == decisions).mean().round(2)

np.float64(0.7)

## Question 5
- Let's find the least useful feature using the _feature elimination_ technique.
- Train a model using the same features and parameters as in Q4 (without rounding).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

In [255]:
features_to_check = ['industry', 'employment_status', 'lead_score']

In [256]:
def train_model_without_feature_and_get_accuracy(feature_to_exclude):
    dv = DictVectorizer(sparse=False)
    features = [x for x in categorical + numerical if x != feature_to_exclude]
    X_train = dv.fit_transform(df_train[features].to_dict(orient='records'))
    X_val = dv.transform(df_val[features].to_dict(orient='records'))
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:,1]
    decisions = (y_pred >= 0.5)
    return (y_val == decisions).mean()

accuracy = train_model_without_feature_and_get_accuracy(None)
print(f'current: {accuracy}')
print()

for f in features_to_check:
    print(f)
    current_accuracy = train_model_without_feature_and_get_accuracy(f)
    print(f'current: {current_accuracy}')
    print(f'diff: {abs(accuracy - current_accuracy)}')
    print()

current: 0.6996587030716723

industry
current: 0.6996587030716723
diff: 0.0

employment_status
current: 0.6962457337883959
diff: 0.0034129692832763903

lead_score
current: 0.7064846416382252
diff: 0.0068259385665528916



## Question 6
- Now let's train a regularized logistic regression.
- Let's try the following values of the parameter `C`: `[0.01, 0.1, 1, 10, 100]`.
- Train models using all the features as in Q4.
- Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these `C` leads to the best accuracy on the validation set?

In [257]:
def train_model_with_c_and_get_accuracy(reg):
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(df_train.to_dict(orient='records'))
    X_val = dv.transform(df_val.to_dict(orient='records'))
    model = LogisticRegression(solver='liblinear', C=reg, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:,1]
    decisions = (y_pred >= 0.5)
    return (y_val == decisions).mean()

for c in [0.01, 0.1, 1, 10, 100]:
    print(f'current C: {c}')
    print(f'accuracy: {train_model_with_c_and_get_accuracy(c).round(3)}')
    print()

current C: 0.01
accuracy: 0.7

current C: 0.1
accuracy: 0.7

current C: 1
accuracy: 0.7

current C: 10
accuracy: 0.7

current C: 100
accuracy: 0.7

